In [1]:
!pip install spotipy

  Running setup.py bdist_wheel for spotipy ... - done
  Stored in directory: /root/.cache/pip/wheels/76/28/19/a86ca9bb0e32dbd4a4f580870250f5aeef852870578e0427e6
Successfully built spotipy


In [0]:
import pandas as pd
import nltk
import spotipy
import spotipy.util as util
import json
import numpy as np

In [0]:
key_file = 'keys.json'
keys = json.load(open(key_file))

SPOTIPY_CLIENT_ID= keys["client_id"]
SPOTIPY_CLIENT_SECRET= keys["client_secret"]
SPOTIPY_REDIRECT_URI= keys["redirect_url"]

In [6]:
token = util.prompt_for_user_token("1274418332", 
                                   scope = "user-library-read", 
                                   client_id=SPOTIPY_CLIENT_ID, 
                                   client_secret=SPOTIPY_CLIENT_SECRET, 
                                   redirect_uri = SPOTIPY_REDIRECT_URI)
spotify = spotipy.Spotify(auth=token)



            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=b29ebdb5e8ac4efaa4e1afe0da5a7e7c&response_type=code&redirect_uri=http%3A%2F%2Fgoogle.com.mx%2F&scope=user-library-read in your browser


Enter the URL you were redirected to: https://www.google.com.mx/?code=AQAD3RUb_Xpoqfv2vtUx8G3bbt24BrkG6jwR2Bqyv0eIkgnE0H0zLEThPNBHj1I8YY3O7Q1CE8lPlSgJ7v6I8t2nAnCHDv79wcelJa8EyJ85CSr_miJtolmLPiVXsWpCL_lq05MJTZH2SD_f_WWyHl7KxJSPamZCJkxyGMzkWGAO1YRnuiGQYEdib-w6w3VCBaf0D6gs_HB2X9TmkeLNEQ




In [7]:
billboard = pd.read_csv("billboard_lyrics_1964-2015.csv")
billboard.head()

,Rank,Song,Artist,Year,Lyrics,Source
0,1,wooly bully,sam the sham and the pharaohs,1965,sam the sham miscellaneous wooly bully wooly b...,3.0
1,2,i cant help myself sugar pie honey bunch,four tops,1965,sugar pie honey bunch you know that i love yo...,1.0
2,3,i cant get no satisfaction,the rolling stones,1965,,1.0
3,4,you were on my mind,we five,1965,when i woke up this morning you were on my mi...,1.0
4,5,youve lost that lovin feelin,the righteous brothers,1965,you never close your eyes anymore when i kiss...,1.0


In [0]:
def get_valence():
    try:
        song = spotify.search(q='artist:' + artist + ' track:' + song, type='track')
        song_uri = song['tracks']['items'][0]['uri']
        song_id = song_uri.split('spotify:track:')[1]
        song_features = spotify.audio_features(song_id)
        valence = song_features[0]['valence']
        print('success: ' + str(year))
        return valence
    except IndexError as error:
        print('fail')
        return np.nan

In [0]:
billboard['Query'] = 'artist: ' + billboard['Artist'] + ', track: ' + billboard['Song']

In [0]:
sixties = billboard[billboard['Year'] < 1970]
seventies = billboard[(billboard['Year'] >= 1970) & (billboard['Year'] < 1980)]
eighties = billboard[(billboard['Year'] >= 1980) & (billboard['Year'] < 1990)]
nineties = billboard[(billboard['Year'] >= 1990) & (billboard['Year'] < 2000)]
thousands = billboard[(billboard['Year'] >= 2000) & (billboard['Year'] < 2010)]
twenty_tens = billboard[(billboard['Year'] >= 2010) & (billboard['Year'] < 2020)]

In [52]:
twenty_tens.head()

,Rank,Song,Artist,Year,Lyrics,Source,Query
4500,1,tik tok,keha,2010,wake up in the morning feeling like p diddy h...,1.0,"artist: keha, track: tik tok"
4501,2,need you now,lady antebellum,2010,picture perfect memories scattered all around...,1.0,"artist: lady antebellum, track: need you now"
4502,3,hey soul sister,train,2010,hey hey hey hey hey heyyour lipstick stains o...,1.0,"artist: train, track: hey soul sister"
4503,4,california gurls,katy perry featuring snoop dogg,2010,snoop dogg greetings loved ones lets take a j...,1.0,"artist: katy perry featuring snoop dogg, track..."
4504,5,omg,usher featuring william,2010,oh my gosh baby let me did it again so imma l...,1.0,"artist: usher featuring william, track: omg"


In [55]:
song = spotify.search(q= twenty_tens['Query'][4500], type='track')
song_uri = song['tracks']['items'][0]['uri']

IndexError: ignored

In [54]:
song_uri

{'href': 'https://api.spotify.com/v1/search?query=artist%3A+keha%2C+track%3A+tik+tok&type=track&market=US&offset=0&limit=10',
 'items': [],
 'limit': 10,
 'next': None,
 'offset': 0,
 'previous': None,
 'total': 0}